In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
print(tf.__version__)
print(np.__version__)


2.0.0
1.18.1


# Dataset类



# Dataset类读取numpy数据
最基础的建立 tf.data.Dataset 的方法是使用 tf.data.Dataset.from_tensor_slices() ，适用于数据量较小（能够整个装进内存）的情况。

具体而言，如果我们的数据集中的所有元素通过张量的第 0 维，拼接成一个大的张量（例如，前节的 MNIST 数据集的训练集即为一个 [60000, 28, 28, 1] 的张量，表示了 60000 张 28*28 的单通道灰度图像），那么我们提供一个这样的张量或者第 0 维大小相同的多个张量作为输入，即可按张量的第 0 维展开来构建数据集，数据集的元素数量为张量第 0 位的大小。

In [ ]:
mnist = np.load("mnist.npz")

x_train, y_train = mnist['x_train'],mnist['y_train']

In [ ]:
x_train.shape,y_train.shape

In [ ]:
x_train = np.expand_dims(x_train, axis=-1) 

In [ ]:
x_train.shape

In [ ]:
mnist_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))

In [ ]:
for image, label in mnist_dataset:
    plt.title(label.numpy())
    plt.imshow(image.numpy()[:, :,0])
    plt.show()
    break

## Pandas数据读取

In [ ]:
import pandas as pd
df = pd.read_csv('heart.csv')

In [ ]:
df.head()

In [ ]:
df.dtypes

In [ ]:
df['thal'] = pd.Categorical(df['thal'])
df['thal'] = df.thal.cat.codes

In [ ]:
target = df.pop('target')

In [ ]:
dataset = tf.data.Dataset.from_tensor_slices((df.values, target.values))

In [ ]:
for feat, targ in dataset.take(5):
    print ('Features: {}, Target: {}'.format(feat, targ))

In [2]:
img_gen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255, rotation_range=20)

In [3]:
flowers = './flower_photos/flower_photos/'

In [4]:
images, labels = next(img_gen.flow_from_directory(flowers))

Found 3670 images belonging to 5 classes.


In [5]:
print(images.dtype, images.shape)
print(labels.dtype, labels.shape)

float32 (32, 256, 256, 3)
float32 (32, 5)


In [7]:
ds = tf.data.Dataset.from_generator(
    img_gen.flow_from_directory, args=[flowers], 
    output_types=(tf.float32, tf.float32), 
    output_shapes=([32,256,256,3], [32,5])
)

In [ ]:
for image,label in ds:
    print(image.shape,label.shape)
    break

# TFRecordDataset类

In [7]:
feature_description = { # 定义Feature结构，告诉解码器每个Feature的类型是什么
    'image': tf.io.FixedLenFeature([], tf.string),
    'label': tf.io.FixedLenFeature([], tf.int64),
}

def _parse_example(example_string): # 将 TFRecord 文件中的每一个序列化的 tf.train.Example 解码
    feature_dict = tf.io.parse_single_example(example_string, feature_description)
    feature_dict['image'] = tf.io.decode_jpeg(feature_dict['image'])    # 解码JPEG图片
    feature_dict['image'] = tf.image.resize(feature_dict['image'], [256, 256]) / 255.0
    return feature_dict['image'], feature_dict['label']

batch_size = 32

train_dataset = tf.data.TFRecordDataset("sub_train.tfrecords")    # 读取 TFRecord 文件
train_dataset = train_dataset.map(_parse_example)


In [ ]:
for line in train_dataset:
    print(line)
    break

# TextLineDataset类

In [2]:
titanic_lines = tf.data.TextLineDataset(['train.csv','eval.csv'])

In [3]:
def data_func(line):
    line = tf.strings.split(line, sep = ",")
    return line

In [4]:
titanic_data = titanic_lines.skip(1).map(data_func)

In [5]:
for line in titanic_data:
    print(line)
    break

tf.Tensor(
[b'0' b'male' b'22.0' b'1' b'0' b'7.25' b'Third' b'unknown' b'Southampton'
 b'n'], shape=(10,), dtype=string)
